In [1]:
from bs4 import BeautifulSoup
import requests
#import BeautifulSoup import csv 
import pandas as pd
import numpy as np
import lxml
import warnings
import time
import re
import string

In [10]:
warnings.filterwarnings('ignore')

### Pruebas iniciales de conexión y extracción de datos

In [26]:
response = requests.get("https://finance.yahoo.com/lookup/etf?s=vxus")
#soup = BeautifulSoup(r.text, ‘lxml’)
#soup = BeautifulSoup(response.text,'lxml')
soup = BeautifulSoup(response.content,  "html.parser")
price = soup.select('table td')
print(price[2].text)

54.56


In [29]:
price[2].text

'54.56'

In [35]:
response = requests.get("https://finance.yahoo.com/quote/ELE.MC/key-statistics?p=ELE.MC")
#soup = BeautifulSoup(r.text, ‘lxml’)
#soup = BeautifulSoup(response.text,'lxml')
soup = BeautifulSoup(response.content,  "html.parser")
statistics = soup.select('table td')
#print(price[2].text)
Market_Cap = statistics[1].text
print(Market_Cap)

26.77B


In [50]:
statistics[0]

<td class="Pos(st) Start(0) Bgc($lv2BgColor) ks-row:h_Bgc($extraLightBlue) Pend(10px) Miw(140px)" data-reactid="15"><span data-reactid="16">Market Cap (intraday)</span><!-- react-text: 17 --> <!-- /react-text --><!-- react-text: 18 --><!-- /react-text --><sup aria-label="Shares outstanding is taken from the most recently filed quarterly or annual report and Market Cap is calculated using shares outstanding." data-reactid="19">5</sup></td>

In [88]:
index = range(100)
df = pd.DataFrame(index=index)
for field in range(0, len(statistics),2):
    df[statistics[field].text] = ''
df

,Market Cap (intraday) 5,Enterprise Value 3,Trailing P/E,Forward P/E 1,PEG Ratio (5 yr expected) 1,Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue 3,Enterprise Value/EBITDA 6,Beta (5Y Monthly),...,Diluted EPS (ttm),Quarterly Earnings Growth (yoy),Total Cash (mrq),Total Cash Per Share (mrq),Total Debt (mrq),Total Debt/Equity (mrq),Current Ratio (mrq),Book Value Per Share (mrq),Operating Cash Flow (ttm),Levered Free Cash Flow (ttm)
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,,,,,,,,,,,...,,,,,,,,,,
96,,,,,,,,,,,...,,,,,,,,,,
97,,,,,,,,,,,...,,,,,,,,,,
98,,,,,,,,,,,...,,,,,,,,,,


In [82]:
df.columns[0]

'Market Cap (intraday) 5'

In [78]:
statistics[1].text

'26.77B'

In [147]:
for i in range(len(df.columns)):
    print(df.columns[i])
    print(statistics[2*i+1].text)
    df[df.columns[i]][0] = statistics[2*i+1].text
    

Market Cap (intraday) 5
26.77B
Enterprise Value 3
34.69B
Trailing P/E 
67.06
Forward P/E 1
18.19
PEG Ratio (5 yr expected) 1
6.91
Price/Sales (ttm)
1.39
Price/Book (mrq)
3.21
Enterprise Value/Revenue 3
1.80
Enterprise Value/EBITDA 6
9.98
Beta (5Y Monthly) 
0.42
52-Week Change 3
17.35%
S&P500 52-Week Change 3
20.64%
52 Week High 3
26.26
52 Week Low 3
21.57
50-Day Moving Average 3
24.59
200-Day Moving Average 3
24.05
Avg Vol (3 month) 3
1.37M
Avg Vol (10 day) 3
1.12M
Shares Outstanding 5
1.06B
Float 
316.51M
% Held by Insiders 1
N/A
% Held by Institutions 1
N/A
Shares Short  4
N/A
Short Ratio  4
N/A
Short % of Float  4
N/A
Short % of Shares Outstanding  4
N/A
Shares Short (prior month ) 4
N/A
Forward Annual Dividend Rate 4
1.43
Forward Annual Dividend Yield 4
5.53%
Trailing Annual Dividend Rate 3
1.43
Trailing Annual Dividend Yield 3
5.52%
5 Year Average Dividend Yield 4
5.73
Payout Ratio 4
376.09%
Dividend Date 3
N/A
Ex-Dividend Date 4
Dec 30, 2019
Last Split Factor 2
N/A
Last Split Dat

In [152]:
df[:1]

,Market Cap (intraday) 5,Enterprise Value 3,Trailing P/E,Forward P/E 1,PEG Ratio (5 yr expected) 1,Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue 3,Enterprise Value/EBITDA 6,Beta (5Y Monthly),...,Diluted EPS (ttm),Quarterly Earnings Growth (yoy),Total Cash (mrq),Total Cash Per Share (mrq),Total Debt (mrq),Total Debt/Equity (mrq),Current Ratio (mrq),Book Value Per Share (mrq),Operating Cash Flow (ttm),Levered Free Cash Flow (ttm)
0,26.77B,34.69B,67.06,18.19,6.91,1.39,3.21,1.80,9.98,0.42,...,0.38,N/A,495M,0.47,7.67B,90.34,0.78,7.87,3.09B,1.56B


## Obtener acrónimos/tickers Compañias Bolsa de Madrid

In [166]:
#'https://es.finance.yahoo.com/quote/IGBM.MA/components?p=IGBM.MA'
response = requests.get('https://es.finance.yahoo.com/quote/IGBM.MA/components?p=IGBM.MA')
soup = BeautifulSoup(response.content,  "html.parser")
data = soup.select('table td')
#print(price[2].text)
data

[<td class="Py(10px) Ta(start) Pend(10px)" data-reactid="26"><a class="C($c-fuji-blue-1-b) Cur(p) Td(n) Fw(500)" data-reactid="27" href="/quote/ABE.MC?p=ABE.MC" title="ABE.MC">ABE.MC</a></td>,
 <td class="Py(10px) Ta(start) Pend(10px)" data-reactid="28">Abertis Infraestructuras, S.A.</td>,
 <td class="Py(10px) Pstart(10px)" data-reactid="29"></td>,
 <td class="Py(10px) Pstart(10px)" data-reactid="30"></td>,
 <td class="Py(10px) Pstart(10px)" data-reactid="31"><span class="Trsdu(0.3s)" data-reactid="32"></span></td>,
 <td class="Py(10px) Pstart(10px)" data-reactid="33"></td>,
 <td class="Py(10px) Ta(start) Pend(10px)" data-reactid="35"><a class="C($c-fuji-blue-1-b) Cur(p) Td(n) Fw(500)" data-reactid="36" href="/quote/NTH.MC?p=NTH.MC" title="NTH.MC">NTH.MC</a></td>,
 <td class="Py(10px) Ta(start) Pend(10px)" data-reactid="37">Naturhouse Health, S.A.</td>,
 <td class="Py(10px) Pstart(10px)" data-reactid="38">2,2150</td>,
 <td class="Py(10px) Pstart(10px)" data-reactid="39">-0,0250</td>,
 

In [167]:
tikers = soup.text.split('components":{"components":')[1].split('],')[0].split(',')[0:-1]
tikers[:10]

['["ADV.MC"',
 '"BBG000DW1V13"',
 '"GAS.MC"',
 '"DIN.MA"',
 '"ELE.MC"',
 '"EKT.MC"',
 '"APPS.MC"',
 '"CLNX.MC"',
 '"MDF.MC"',
 '"NTH.MC"']

In [233]:
# We created a new dataframe to store the extracted tikers
index = range(len(tikers))
df_tikers = pd.DataFrame(index=index, columns = ['ticker'])
df_tikers.head()

,ticker
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


In [234]:
for i in range(len(tikers)):
    #print('Name:', ScrapedAux[i].replace('"','').replace('[',''))
    tiker = tikers[i].replace('"','').replace('[','')
    df_tikers['ticker'][i] = tiker
df_tikers.head()

,ticker
0,ADV.MC
1,BBG000DW1V13
2,GAS.MC
3,DIN.MA
4,ELE.MC


In [235]:
#We removed some ticker whose format does not match the acronyms used by Yahoo
df_tikers = df_tikers[~(df_tikers['ticker'].astype(str).str.len()>=9)]

In [236]:
df_tikers.reset_index(drop=True, inplace=True)

In [237]:
df_tikers

,ticker
0,ADV.MC
1,GAS.MC
2,DIN.MA
3,ELE.MC
4,EKT.MC
...,...
94,ACS.MC
95,IAG.MC
96,PRM.MC
97,INY.MC


Añadimos los datos estadísticos de cada una de las empresas que corresponde a cada ticker

In [173]:
#Probamos con el 1º ticker
df_tikers['ticker'][10]

'GALQ.MC'

In [174]:
#response = requests.get("https://finance.yahoo.com/quote/ELE.MC/key-statistics?p=ELE.MC")
stock = df_tikers['ticker'][10]
response = requests.get('https://finance.yahoo.com/quote/'+stock+'/key-statistics?p='+stock)
#soup = BeautifulSoup(r.text, ‘lxml’)
#soup = BeautifulSoup(response.text,'lxml')
soup = BeautifulSoup(response.content,  "html.parser")
statistics = soup.select('table td')
statistics[0]

<td class="Pos(st) Start(0) Bgc($lv2BgColor) ks-row:h_Bgc($extraLightBlue) Pend(10px) Miw(140px)" data-reactid="15"><span data-reactid="16">Market Cap (intraday)</span><!-- react-text: 17 --> <!-- /react-text --><!-- react-text: 18 --><!-- /react-text --><sup aria-label="Shares outstanding is taken from the most recently filed quarterly or annual report and Market Cap is calculated using shares outstanding." data-reactid="19">5</sup></td>

In [176]:
statistics[0].get_text()

'Market Cap (intraday) 5'

In [175]:
statistics[1].get_text()

'106.59M'

In [77]:
statistics

[<td class="Pos(st) Start(0) Bgc($lv2BgColor) ks-row:h_Bgc($extraLightBlue) Pend(10px) Miw(140px)" data-reactid="15"><span data-reactid="16">Market Cap (intraday)</span><!-- react-text: 17 --> <!-- /react-text --><!-- react-text: 18 --><!-- /react-text --><sup aria-label="Shares outstanding is taken from the most recently filed quarterly or annual report and Market Cap is calculated using shares outstanding." data-reactid="19">5</sup></td>,
 <td class="Fz(s) Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="20">108.17M</td>,
 <td class="Pos(st) Start(0) Bgc($lv2BgColor) ks-row:h_Bgc($extraLightBlue) Pend(10px)" data-reactid="22"><span data-reactid="23">Enterprise Value</span><!-- react-text: 24 --> <!-- /react-text --><!-- react-text: 25 --><!-- /react-text --><sup aria-label="Data derived from multiple sources or calculated by Yahoo Finance." data-reactid="26">3</sup></td>,
 <td class="Fz(s) Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="27">279.69M</td>,
 <td class="Pos(st

In [238]:
#index = range(len(df_tikers))
#df = pd.DataFrame(index=index)
#Add others fields of statistic webpages
for field in range(0, len(statistics),2):
    df_tikers[statistics[field].text] = ''
#We added two more fields to identify each company
df_tikers['ScrapedName'] = ''
df_tikers['Sector'] = ''
df_tikers = df_tikers[3:]#Los 3 primeros tickers no producen ningún resultado en yahoo
df_tikers.reset_index(drop=True, inplace=True)
df_tikers.to_csv('df_tikers.csv')

In [179]:
#Prueba para el primero de los tikers[0]
for i in range(len(df.columns)):
    #print(df.columns[i])
    #print(statistics[2*i+1].text)
    df[df.columns[i]][0] = statistics[2*i+1].text
df

,Market Cap (intraday) 5,Enterprise Value 3,Trailing P/E,Forward P/E 1,PEG Ratio (5 yr expected) 1,Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue 3,Enterprise Value/EBITDA 6,Beta (5Y Monthly),...,Diluted EPS (ttm),Quarterly Earnings Growth (yoy),Total Cash (mrq),Total Cash Per Share (mrq),Total Debt (mrq),Total Debt/Equity (mrq),Current Ratio (mrq),Book Value Per Share (mrq),Operating Cash Flow (ttm),Levered Free Cash Flow (ttm)
0,107.22M,279.69M,N/A,N/A,N/A,0.70,N/A,1.83,7.84,0.70,...,-0.1180,N/A,9.68M,0.29,229.23M,N/A,N/A,-0.31,N/A,N/A
1,,,,,,,,,,,...,,,,,,,,,,
2,,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,,,,,,,,,,,...,,,,,,,,,,
95,,,,,,,,,,,...,,,,,,,,,,
96,,,,,,,,,,,...,,,,,,,,,,
97,,,,,,,,,,,...,,,,,,,,,,


In [152]:
#Cómo sacar el campo Nombre completo que corresponde al tickers
soup.text.split('Find out all the key statistics for')[1].split(', including')[0]

' {shortName} ({symbol})'

In [30]:
#Extraer los campos Sector e Industria
soup.find_all('span', class_="Fw(600)")

[<span class="Fw(600)" data-reactid="23">Industrials</span>,
 <span class="Fw(600)" data-reactid="27">Aerospace &amp; Defense</span>,
 <span class="Fw(600)" data-reactid="31"><span data-reactid="32">1,060</span></span>]

In [25]:
#response = requests.get("https://finance.yahoo.com/quote/ELE.MC/key-statistics?p=ELE.MC")
stock = df_tikers['ticker'][10]
response = requests.get('https://finance.yahoo.com/quote/'+stock+'/profile?p='+stock)
#soup = BeautifulSoup(r.text, ‘lxml’)
#soup = BeautifulSoup(response.text,'lxml')
soup = BeautifulSoup(response.content,  "html.parser")
#statistics = soup.select('table td')
#statistics[0]
#soup.select('span', 'data-reactid="23"')
soup.find_all('span', class_="Fw(600)")[0].get_text()

'Industrials'

In [53]:
stock = df_tikers['ticker'][10]
ScrapedSector = ''
r = requests.get('https://finance.yahoo.com/quote/'+stock+'/profile?p='+stock)
s = BeautifulSoup(r.content,  "html.parser")
s.find_all('span', class_="Fw(600)")[0].get_text()

'Industrials'

In [189]:
df_tikers['ticker'][10]

'GALQ.MC'

In [159]:
#Para extraer nombre de la compañía mediante split
stock = df_tikers['ticker'][3]
response = requests.get('https://finance.yahoo.com/quote/'+stock+'/key-statistics?p='+stock)
#soup = BeautifulSoup(r.text, ‘lxml’)
#soup = BeautifulSoup(response.text,'lxml')
soup = BeautifulSoup(response.content,  "html.parser")
#Cómo sacar el campo Nombre completo que corresponde al tickers
soup.text.split('Find out all the key statistics for')[1].split(', including')[0][1:]

'ENDESA,S.A. (ELE.MC)'

In [160]:
#Para extraer nombre de la compañía mediante re
s = soup.find_all('meta', {"name":"description"})[0]['content']
#re.split(r'\s+[a-z][a-z\s]*', s)[1]
' '.join(re.findall('[A-Z.()]+',s))[2:-2]
#"".join(c for c in s if c.isupper() or c == ' ')
#re.sub('[^A-Z]', '', s) Selecciona las Mayusculas pero las une todas sin espacios

'ENDESA S.A. (ELE.MC)'

In [12]:
df_tikers = pd.read_csv('df_tikers.csv', index_col=0)

In [13]:
df_tikers

,ticker,Market Cap (intraday) 5,Enterprise Value 3,Trailing P/E,Forward P/E 1,PEG Ratio (5 yr expected) 1,Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue 3,Enterprise Value/EBITDA 6,...,Total Cash (mrq),Total Cash Per Share (mrq),Total Debt (mrq),Total Debt/Equity (mrq),Current Ratio (mrq),Book Value Per Share (mrq),Operating Cash Flow (ttm),Levered Free Cash Flow (ttm),ScrapedName,Sector
0,ELE.MC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EKT.MC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,APPS.MC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CLNX.MC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MDF.MC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,ACS.MC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,IAG.MC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,PRM.MC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,INY.MC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [241]:
df_tikers['ticker'][0]

'ELE.MC'

In [265]:
for i in range(1,len(df_tikers.columns)):
    print(df_tikers.columns[i])
    break

Market Cap (intraday) 5


In [15]:
#Introducimos los datos estadísticos
start_time = time.time()
for j in range(len(df_tikers)):
    stock = df_tikers['ticker'][j]
    response = requests.get('https://finance.yahoo.com/quote/'+stock+'/key-statistics?p='+stock)
    soup = BeautifulSoup(response.content, "html.parser")
    statistics = soup.select('table td')
    for i in range(1, len(df_tikers.columns)):
        field = df_tikers.columns[i]
        try:
            df_tikers[field][j] = statistics[2*i+1].text 
        except:                                                         
            df_tikers[field][j] = np.nan
            continue
print("--- %s minutos  ---" % (round((time.time() - start_time)/60,2)))

--- 1.23 minutos  ---


In [17]:
#Introducimos el nombre de la compañía
start_time = time.time()
for j in range(len(df_tikers)):
    stock = df_tikers['ticker'][j]
    response = requests.get('https://finance.yahoo.com/quote/'+stock+'/key-statistics?p='+stock)
    soup = BeautifulSoup(response.content, "html.parser")
    s = soup.find_all('meta', {"name":"description"})[0]['content']
    try:
        compname = ' '.join(re.findall('[A-Z.()]+',s))[2:-2]
    except:
        compname = soup.text.split('Find out all the key statistics for')[1].split(', including')[0][1:]
        if compname.find('{shortName} ({symbol})') >= 0:
            compname = np.nan
            continue   
    df_tikers['ScrapedName'][j] = compname
print("--- %s minutos  ---" % (round((time.time() - start_time)/60,2)))

--- 1.19 minutos  ---


In [39]:
#Introducción del Sector
start_time = time.time()
for j in range(len(df_tikers)):
    stock = df_tikers['ticker'][j]
    response = requests.get('https://finance.yahoo.com/quote/'+stock+'/profile?p='+stock)
    soup = BeautifulSoup(response.content,  "html.parser")
    try:
        df_tikers['Sector'][j] = soup.find_all('span', {'data-reactid':"21"})[1].get_text()
    except:
        df_tikers['Sector'][j] = np.nan
        continue
print("--- %s minutos  ---" % (round((time.time() - start_time)/60,2)))

--- 1.17 minutos  ---


In [40]:
df_tikers

,ticker,Market Cap (intraday) 5,Enterprise Value 3,Trailing P/E,Forward P/E 1,PEG Ratio (5 yr expected) 1,Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue 3,Enterprise Value/EBITDA 6,...,Total Cash (mrq),Total Cash Per Share (mrq),Total Debt (mrq),Total Debt/Equity (mrq),Current Ratio (mrq),Book Value Per Share (mrq),Operating Cash Flow (ttm),Levered Free Cash Flow (ttm),ScrapedName,Sector
0,ELE.MC,N/A,65.65,17.81,6.91,1.3,3.14,N/A,N/A,N/A,...,N/A,N/A,90.34,N/A,7.87,N/A,N/A,NaN,ENDESA S.A. (ELE.MC),Utilities
1,EKT.MC,2.97B,25.03,18.13,1.56,2.13,1.48,4.34,9.86,0.62,...,0.46,1.58B,160.34,0.43,5.53,N/A,N/A,NaN,EUSKALTEL S.A. (EKT.MC),Communication Services
2,APPS.MC,2.2B,32,14.72,2.35,0.91,2.1,1.26,9.82,0.74,...,0.93,778.72M,96.95,1.36,5.24,200.22M,126.27M,NaN,APPLUS SERVICES S.A. (APPS.MC),Sector
3,CLNX.MC,17.45B,N/A,N/A,N/A,15.57,6.83,18.26,27.23,0.48,...,7.77,5.72B,280.48,4.01,6.83,N/A,N/A,NaN,CELLNEX TELECOM S.A. (CLNX.MC),Communication Services
4,MDF.MC,-71.58M,N/A,N/A,N/A,0.06,4.05,-0.17,0.99,1.52,...,2.14,108.7M,"1,764.26",1.07,0.06,N/A,N/A,NaN,DURO FELGUERA S.A. (MDF.MC),Sector
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,ACS.MC,10.59B,8.84,9.74,6.22,0.22,1.92,0.27,4.17,1.28,...,30.81,10.49B,190.85,1.01,14.44,1.72B,3.06B,NaN,ACS ACTIVIDADES DE CONSTRUCCION (ACS.MC),Industrials
92,IAG.MC,18.21B,6.08,5.56,1.53,0.51,2.07,0.71,4.33,0.85,...,4.03,12.81B,194.65,N/A,3.17,N/A,N/A,NaN,INTL CONSOLIDATED AIRLINES GROU (IAG.MC),Sector
93,PRM.MC,197.94M,16.45,N/A,N/A,1.35,1.84,1.36,10.11,0.27,...,0.59,9.58M,8.85,3.26,6.24,N/A,N/A,NaN,PRIM S.A. (PRM.MC),Sector
94,INY.MC,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,. T Y,Sitemap


In [42]:
df_tikers.to_csv('df_bs4_stocks.csv')